# Air Quality Data Analysis


The goal of this challenge is to analyze a database sourced from [Chinese Real-Time Air Quality Monitoring Platform](http://106.37.208.233:20035/). Some cells are already implemented, you just need to **run** them. Some other cells need you to write some code.


##### Agenda

- Python 101
- Your first Exploratory Data Analysis with Pandas, matplotlib and Seaborn

We will code in a [Notebook](https://jupyter.org/).


##### Here is a quick guide on how to use this Jupyter Notebook 🤔

* Type inside the empty cells to write code. These empty cells will have a `In [ ]:` prefix before
* Press the `return/enter ⏎` key to add a new line inside the cell
* To display your results use the Python built in `print(STUFF_YOU_WANT_TO_PRINT)` method or simply put the stuff you want to print as the last line inside the cell. The result of the last line will appear as the `Out[]:` or the output of the cell :)
* Press `shift` + `return/enter ⏎` to run your code 🤓 this will run the code inside your currently selected cell and print anything inside `print()` method and the last line of your cell
* To add a new cell, select any cell and press the `b` key (make sure you are not just typing the letter `b` in the cell). This will add a new cell below
* To delete a cell, double press the `d` key (make sure you are not just typing the letter `d` in the cell)

##### Start the challenge by running the two following cells:

In [ ]:
# we will need these libraries to run our analytics and visualisation
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## Data processing

In [ ]:
# we will read the CSV file into a DataFrame - the format that we can easily analyze and manipulate
# For example, the original datasets we extract from the Chinese official website
raw_df = pd.read_csv('data/rawdatasets/city_AQI/city_2020/china_cities_20201121.csv', decimal=",")

In [ ]:
raw_df

### 👀 Feel free to have a quick glance at the data - `.shape`, `.columns`, `.head()`, `.tail()`, `.dtypes` 
<br>

<details>
    <summary>Not sure how? Reveal some tips 🙈</summary>

<p> 
<pre>
raw_df.shape
raw_df.columns
raw_df.head() # you can add a number in parentheses for how many first rows you want to display
raw_df.dtypes
</pre>
</details>

### 👀  Air quality data acquired at official monitoring stations nationwide. 
The level of six atmospheric pollutants was reported hourly, and each time an [Air Quality Index (AQI)](https://en.wikipedia.org/wiki/Air_quality_index) would be calculated based on these pollutants data. 

The six atmospheric pollutants include sulfur dioxide ($\rm SO_2$), nitrogen dioxide ($\rm NO_2$), particular matters smaller than 10 $\mu m$ in aerodynamic diameter ($\mathrm{PM_{10}}$), particular matters smaller than 2.5 $\mu m$ ($\rm PM_{2.5}$), carbon monoxide (CO), and ozone ($\rm O_3$). 

An individual score is assigned to each pollutant and the final AQI is the highest of these six scores.





In [ ]:
items = pd.read_csv('data/measurement_item_list.csv')

In [ ]:
print('Shape:', items.shape)
items

### Parsing data and creating auxiliar structures

Data preprocessing usually takes most the time in a project. According to CrowdFlower 2016 report, data scientists in average take 60% of their time to clean and organise data. For our limited time in this session, we have a pre-processed dataframe for you already (We translated the name of the city stations from Chinese to English, extracted only part of the pollutant readings, and put the daily recording of almost three years into a single [pickle](https://docs.python.org/3/library/pickle.html#data-stream-format) file.

Now we will read the pickle file into a DataFrame.

In [ ]:
air_df = pd.read_pickle('data/pollutants6_2018-current.pkl', compression='bz2')

In [ ]:
# To examine the DataFrame

__We still have some cleaning to do on our DataFrame.__ 🧹 

First of all, let's extract only AQI data from the master dataframe


__There are missing timestamps in your dataframe. Before moving to further analysis, shall we fill them up with NaN (not a number) so that our dataframe will have a continuous hourly reading.__  

<details>
    <summary>Not sure how? Reveal some tips 🙈</summary>

<p> 
<pre>
idx = pd.date_range(min(AQI_df.index), max(AQI_df.index), freq='h')
AQI_df.reindex(idx, fill_value=np.nan)
</pre>
</details>

#####  Let's focus on only 5 cities

So pick up 5 cities of your interest. Write them into a python list. 

Now extract AQI data for your chosen cities from the AQI master dataframe

##### Handling missing Data

There are a few missing data in the AQI readings for each city. You can see it as NaN (not a number) in the dataframe. In the wild, of course, you will experience missing data in different representations. The reasons for missing data might be failure of measurement or programming error. Sometimes, you can simply drop these missing data if they appear unnecessary BUT in this case we want to keep a uniform timestamp and hence we will impute a value to it.

It needs thorough consideration to choose an imputer or write your own imputing programme. This concept may be a bit too advanced to put here. Therefore, we will have the answers ready for you to run. 

In [ ]:
# There are missing AQI values for different locations. Can you see how many of them in total for each city?


For the time series nature of our data, we will use a time interpolation imputer coded as below. Now try to apply it to your DataFrame.

In [ ]:
def time_interpolation_impute(df):
    cities = df.columns.unique()
    for city in cities:
        if city !='index' and city != 'date' and city != 'hour' and city != 'type':
            if df[city].isnull().sum() > 0:
                df[city].interpolate(method='time', inplace=True)

    return df

## Exploratory data analysis

### Create visulisations to have an overview on the trend of pollution levels varing with time

As it's probably new to you, let's try a single lineplot first.

In [ ]:
# Change the dataframe name to your own.
x = myAQI_df.index
y = myAQI_df['Shanghai']

plt.plot(x, y, 'g-', linewidth=2)

Now plot trends for all 5 cities. You can use [subplots](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.subplots.html) for a neat presentation. 

<details>
    <summary>Hint:</summary>

<p> 
<pre>
Try sns.lineplot for the seaborn library.

You can plot for each city individually or use plt.subplots to put them neatly together.
</pre>
</details>

### The percentage of the severely polluted hours 

As we can see above, AQI values change over the location and along the time. AQI tells the level of pollution. In China, six categories ranging from Excellent to Severely Polluted have been designed based on the AQI values.

##### Binning
Here, we want to categorize the AQI values into the 6 levels and visulize them in a bar chart to see the percentage of 'bad' days in the 5 cities, respectively.

👇 Using Panda's `cut` ([doc](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.cut.html)), for each city, bin the data into:
    - Excellent (AQI 0 to 50 included)
    - Good (51 to 100 included)
    - Lightly Polluted (101 to 150 included)
    - Moderately Polluted (151 to 200 included)
    - Heavily Polluted (201 to 300 included)
    - Severely Polluted (300 and above)

#####  Bar chart to see the counts of each pollution level along the past three years.

Using Panda's `value_counts`([doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.value_counts.html)), count the number of appearance of each pollution level for every city

### The Pollutant Distribution on Map

Reported concentrations of pollutants can vary depending on the location. In order to have a better overview of the pollutant forming and movement, we will use geological info to map the pollutant levels to their stations.  

In [ ]:
# Extract the PM2.5 measurement from the air_df and create a working dataframe


In [ ]:
# Load an external dataset containing the geometry data for these measuring stations
geo = pd.read_csv('data/geo_distance.csv')
geo.set_index('city', inplace=True)

In [ ]:
# Map the stations using plotly package
import plotly.express as px



##### We can even visulise the transport of the pollutants over the time and space using plotly.




In [ ]:
# One month data only, considering the computational time. Let's take last December as an example.


In [ ]:
# Reshape the dataframe 


In [ ]:
# Join with geo dataset


In [ ]:
# Plot the motion pictures using plotly.express.density_mapbox
